In [82]:
def sigma(self, n, formal=False, embedding=None, type='point', return_polynomial=False, chow=False):
    from sage.rings.fraction_field import (FractionField, is_FractionField, FractionField_1poly_field)
    from sage.rings.polynomial.multi_polynomial_ring_base import is_MPolynomialRing
    from sage.rings.polynomial.polynomial_ring import is_PolynomialRing
    from sage.rings.polynomial.flatten import FlatteningMorphism, UnflatteningMorphism
    n = ZZ(n)
    if n < 1:
        raise ValueError("period must be a positive integer")
    dom = self.domain()
    if not is_ProjectiveSpace(dom):
        raise NotImplementedError("not implemented for subschemes")
    if not embedding is None:
        from sage.misc.superseded import deprecation
        deprecation(23333, "embedding keyword no longer used")
    if self.degree() <= 1:
        raise TypeError("must have degree at least 2")
    if not type in ['point', 'cycle']:
        raise ValueError("type must be either point or cycle")
    if dom.dimension_relative() > 1 or return_polynomial:
        if type == 'cycle':
            raise NotImplementedError('cycle not implemented for dimension greater than 1')
        base_ring = self.base_ring()
        d = self.degree()
        N = dom.dimension_relative()
        Fn = self.nth_iterate_map(n)
        if not base_ring.is_field():
            F = FractionField(base_ring)
            Fn.normalize_coordinates()
            X = Fn.periodic_points(1, minimal=False, formal=formal, return_scheme=True)
            X = X.change_ring(F)
        else:
            F = base_ring
            if is_FractionField(base_ring):
                if is_MPolynomialRing(base_ring.ring()) or is_PolynomialRing(base_ring.ring()):
                    Fn.normalize_coordinates()
                    Fn_ring = Fn.change_ring(base_ring.ring())
                    X = Fn_ring.periodic_points(1, minimal=False, formal=formal, return_scheme=True)
                    X = X.change_ring(F)
            else:
                X = Fn.periodic_points(1, minimal=False, formal=formal, return_scheme=True)
        print('X:', X)
        print('F:', F)
        newR_unordered = PolynomialRing(base_ring, 'w, t', 2)
        newR = newR_unordered.change_ring(order='lex')
        if chow:
            # create full polynomial ring
            R_unordered = PolynomialRing(base_ring, 'v', 2*N+3)
            R = R_unordered.change_ring(order='lex')
            var = list(R.gens())
            # create polynomial ring for result
            R2 = PolynomialRing(base_ring, var[:N] + var[-2:])
            psi = R2.hom(N*[0]+list(newR.gens()), newR)
            # create substition to set extra variables to 0
            R_zero = {R.gen(N):1}
            for j in range(N+1, 2*N+1):
                R_zero[R.gen(j)] = 0
            t = var.pop()
            w = var.pop()
            var = var[:N]
        else:
            R_unordered = PolynomialRing(base_ring, 'v', N+2)
            R = R_unordered.change_ring(order='lex')
            psi = R_unordered.hom(N*[0] + list(newR_unordered.gens()), newR_unordered)
            var = list(R.gens())
            t = var.pop()
            w = var.pop()
        flattened = False
        if is_FractionField(F) and F!=base_ring:
            if is_PolynomialRing(F.ring()) or is_MPolynomialRing(F.ring()):
                flattened = True
                flat_phi = FlatteningMorphism(R)
                flatR_unordered = flat_phi.codomain()
                Id = matrix.identity(len(flatR_unordered.gens()) - len(F.gens()))
                Id2 = matrix.identity(len(F.gens()))
                from sage.matrix.special import block_matrix
                b = block_matrix([[ZZ(0), Id],[Id2, ZZ(0)]])
                from sage.rings.polynomial.term_order import TermOrder
                flatR = flatR_unordered.change_ring(order=TermOrder(b))
                unflat_phi = UnflatteningMorphism(flatR_unordered, R)
                print('flatR ordering:', flatR.term_order())
        print('R ordering:', R.term_order())
        sigma_polynomial = 1
        # go through each affine patch to avoid repeating periodic points
        # setting the visited coordiantes to 0 as we go
        for j in range(N,-1,-1):
            Xa = X.affine_patch(j)
            print('Xa:', Xa)
            fa = Fn.dehomogenize(j)
            Pa = fa.domain()
            Ra = Pa.coordinate_ring()
            # create the images for the Hom to the ring we will do the elimination over
            # with done affine patch coordinates as 0
            if chow:
                im = [R.gen(i) for i in range(j)] + (N-j)*[0] + [R.gen(i) for i in range(N, R.ngens())]
            else:
                im = list(R.gens())[:j] + (N-j)*[0] + [R.gen(i) for i in range(N, R.ngens())]
            phi = Ra.hom(R.gens()[0:len(Ra.gens())])
            # create polymomial that evaluates to the characteristic polynomial
            M = t*matrix.identity(R, N)
            g = (M-jacobian([phi(F.numerator())/phi(F.denominator()) for F in fa], var)).det()
            print('g:', g)
            # create the terms of the sigma invariants prod(w-lambda)
            g_prime = w*R(g.denominator())(im)-R(g.numerator())(im)
            print('g_prime:', g_prime)
            # move the defining polynomials to the polynomial ring
            L = [phi(h)(im) for h in Xa.defining_polynomials()]
            print('L1:', L)
            # add the appropriate final polynomial to compute the sigma invariant polynomial
            # via a Poisson product in elimination
            if chow:
                L += [g_prime + sum(R.gen(j-1)*R.gen(N+j)*(R(g.denominator())(im)) for j in range(1,N+1))]
            else:
                L += [g_prime]
            I = R.ideal(L)
            if flattened:
                I = flatR.ideal([flat_phi(poly) for poly in I.gens()])
            # since R is lex ordering, this is an elimination step
            print('I:', I)
            G = I.groebner_basis()
            print('G:', G)
            print('G[-1]:', G[-1])
            # the polynomial we need is the one just in w and t
            if chow:
                if flattened:
                    unflattened = unflat_phi(G[-1])
                else:
                    unflattened = G[-1]
                poly = psi(unflattened.specialization(R_zero))
                if len(list(poly)) > 0 and base_ring.is_field():
                    poly *= poly.coefficients()[0].inverse_of_unit()
                sigma_polynomial *= poly
            else:
                if flattened:
                    sigma_polynomial *= psi(unflat_phi(flatR_unordered(G[-1])))
                else:
                    sigma_polynomial *= psi(G[-1])
        if return_polynomial:
            return sigma_polynomial
        # if we are returing a numerical list, read off the coefficients
        # in order of degree adjusting sign appropriately
        sigmas = []
        sigma_dictionary = dict([list(reversed(i)) for i in list(sigma_polynomial)])
        degree_w, degree_t = sigma_polynomial.degrees()
        w, t = sigma_polynomial.variables()
        sigmas += [degree_w, degree_t]
        for i in range(degree_w+1):
            for j in range(degree_t+1):
                sigmas.append((-1)**(i+j)*sigma_dictionary.pop(w**(degree_w - i)*t**(degree_t - j), 0))
        return sigmas

In [83]:
R.<c>=QQ[]
P.<x,y>=ProjectiveSpace(R,1)
f=DynamicalSystem([x^2+c*y^2,y^2])
sigma(f, 1, chow=True, return_polynomial=True)

X: Closed subscheme of Projective Space of dimension 1 over Fraction Field of Univariate Polynomial Ring in c over Rational Field defined by:
  x^2*y - x*y^2 + c*y^3
F: Fraction Field of Univariate Polynomial Ring in c over Rational Field
flatR ordering: Matrix term order with matrix
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]
[1 0 0 0 0 0]
R ordering: Lexicographic term order
Xa: Closed subscheme of Affine Space of dimension 1 over Fraction Field of Univariate Polynomial Ring in c over Rational Field defined by:
  x^2 - x + c
g: (-2)*v0 + v4
g_prime: 2*v0 + v3 - v4
L1: [v0^2 - v0 + c]
I: Ideal (v0^2 - v0 + c, v0*v2 + 2*v0 + v3 - v4) of Multivariate Polynomial Ring in c, v0, v1, v2, v3, v4 over Rational Field
G: [v0^2 - v0 + c, v0*v2 + 2*v0 + v3 - v4, v0*v3 - v0*v4 - c*v2 - v3 + v4 - 2*c, c*v2^2 + v2*v3 - v2*v4 + 4*c*v2 + v3^2 - 2*v3*v4 + 2*v3 + v4^2 - 2*v4 + 4*c]
G[-1]: c*v2^2 + v2*v3 - v2*v4 + 4*c*v2 + v3^2 - 2*v3*v4 + 2*v3 + v4^2 - 2*v4 + 4*c
Xa: Closed sub

w^3 + (-3)*w^2*t + 2*w^2 + 3*w*t^2 + (-4)*w*t + 4*c*w - t^3 + 2*t^2 + (-4*c)*t

In [8]:
R.<c>=QQ[]
P.<x,y>=ProjectiveSpace(FractionField(R),1)
f=DynamicalSystem([x^2+c*y^2,y^2])
f.sigma_invariants(2, formal=True, return_polynomial=True)

w^4 + (-4)*w^3*t + (-4*c + 8)*w^3 + 6*w^2*t^2 + (12*c - 24)*w^2*t + (-16*c^2 - 16*c + 16)*w^2 + (-4)*w*t^3 + (-12*c + 24)*w*t^2 + (32*c^2 + 32*c - 32)*w*t + (64*c^3 + 64*c^2)*w + t^4 + (4*c - 8)*t^3 + (-16*c^2 - 16*c + 16)*t^2 + (-64*c^3 - 64*c^2)*t

In [13]:
P.<x,y>=ProjectiveSpace(QQ,1)
f=DynamicalSystem([x^2+3*y^2,y^2])
f.sigma_invariants(2, formal=True)
f.periodic_points(2, formal=True, return_scheme=True).rational_points(F=QQbar)

[(-0.50000000000000000? - 1.936491673103709?*I : 1),
 (-0.50000000000000000? + 1.936491673103709?*I : 1)]

In [19]:
P.<x,y>=ProjectiveSpace(QQ,1)
f=DynamicalSystem([x^2+3*y^2,x*y])
sigma(f, 1, chow=True, return_polynomial=True)

X: Closed subscheme of Projective Space of dimension 1 over Rational Field defined by:
  3*y^3
Xa: Closed subscheme of Affine Space of dimension 1 over Rational Field defined by:
  3
g: (v0^2*v4 - v0^2 + 3)/v0^2
g_prime: v0^2*v3 - v0^2*v4 + v0^2 - 3
L1: [3]
I: Ideal (3, v0^3*v2 + v0^2*v3 - v0^2*v4 + v0^2 - 3) of Multivariate Polynomial Ring in v0, v1, v2, v3, v4 over Rational Field
G: [1]
G[-1]: 1
Xa: Closed subscheme of Affine Space of dimension 1 over Rational Field defined by:
  3*y^3
g: (9*v0^4*v4 + 6*v0^2*v4 + 3*v0^2 + v4 - 1)/(9*v0^4 + 6*v0^2 + 1)
g_prime: v3 - v4 + 1
L1: [0]
I: Ideal (0, v0*v2 + v3 - v4 + 1) of Multivariate Polynomial Ring in v0, v1, v2, v3, v4 over Rational Field
G: [v0*v2 + v3 - v4 + 1]
G[-1]: v0*v2 + v3 - v4 + 1


w - t + 1

In [15]:
R.<c>=QQ[]
P.<x,y>=ProjectiveSpace((R),1)
f=DynamicalSystem([x^2+c*y^2,x*y])
sigma(f, 1, chow=True, return_polynomial=True)

X: Closed subscheme of Projective Space of dimension 1 over Fraction Field of Univariate Polynomial Ring in c over Rational Field defined by:
  c*y^3
Xa: Closed subscheme of Affine Space of dimension 1 over Fraction Field of Univariate Polynomial Ring in c over Rational Field defined by:
  c
g: (v0^2*v4 - v0^2 + c)/v0^2
g_prime: v0^2*v3 - v0^2*v4 + v0^2 - c
L1: [c]
I: Ideal (c, v0^3*v2 + v0^2*v3 - v0^2*v4 + v0^2 - c) of Multivariate Polynomial Ring in c, v0, v1, v2, v3, v4 over Rational Field
G: [v0^3*v2 + v0^2*v3 - v0^2*v4 + v0^2, c]
G[-1]: c
Xa: Closed subscheme of Affine Space of dimension 1 over Fraction Field of Univariate Polynomial Ring in c over Rational Field defined by:
  c*y^3
g: (c^2*v0^4*v4 + 2*c*v0^2*v4 + c*v0^2 + v4 - 1)/(c^2*v0^4 + 2*c*v0^2 + 1)
g_prime: v3 - v4 + 1
L1: [0]
I: Ideal (0, v0*v2 + v3 - v4 + 1) of Multivariate Polynomial Ring in c, v0, v1, v2, v3, v4 over Rational Field
G: [v0*v2 + v3 - v4 + 1]
G[-1]: v0*v2 + v3 - v4 + 1


c*w + (-c)*t + c

In [20]:
P.<x,y>=ProjectiveSpace(QQ,1)
f=DynamicalSystem([x^2+3*y^2,x*y])
f.periodic_points(1)

[(1 : 0)]

In [26]:
R.<v0,v1,v2,v3,c> = QQ[]
R(v1^2 - 2*v1*v2 + 2*v1 + v2^2 - 2*v2 + 4*c)

v1^2 - 2*v1*v2 + v2^2 + 2*v1 - 2*v2 + 4*c

In [43]:
K.<c> = QQ[]
F = FractionField(K)
R = PolynomialRing(F, 'v', 3)
v0, v1, v2 = R.gens()
R = R.change_ring(order = 'lex')
I = R.ideal((v0^2 - v0 + c, v1 - v2 + 2*v0))
I.groebner_basis()

[v0 + 1/2*v1 + (-1/2)*v2, v1^2 + (-2)*v1*v2 + 2*v1 + v2^2 + (-2)*v2 + 4*c]

In [42]:
def sigma(self, n, formal=False, embedding=None, type='point', return_polynomial=False, chow=False):
    from sage.rings.fraction_field import (FractionField, is_FractionField, FractionField_1poly_field)
    from sage.rings.polynomial.multi_polynomial_ring_base import is_MPolynomialRing
    from sage.rings.polynomial.polynomial_ring import is_PolynomialRing
    from sage.rings.polynomial.flatten import FlatteningMorphism, UnflatteningMorphism
    n = ZZ(n)
    if n < 1:
        raise ValueError("period must be a positive integer")
    dom = self.domain()
    if not is_ProjectiveSpace(dom):
        raise NotImplementedError("not implemented for subschemes")
    if not embedding is None:
        from sage.misc.superseded import deprecation
        deprecation(23333, "embedding keyword no longer used")
    if self.degree() <= 1:
        raise TypeError("must have degree at least 2")
    if not type in ['point', 'cycle']:
        raise ValueError("type must be either point or cycle")
    if dom.dimension_relative() > 1 or return_polynomial:
        if type == 'cycle':
            raise NotImplementedError('cycle not implemented for dimension greater than 1')
        base_ring = self.base_ring()
        d = self.degree()
        N = dom.dimension_relative()
        f = copy(self)
        Fn = f.nth_iterate_map(n)
        print('formal:', formal)
        if not base_ring.is_field():
            F = FractionField(base_ring)
            f.normalize_coordinates()
            X = f.periodic_points(n, minimal=False, formal=formal, return_scheme=True)
            X = X.change_ring(F)
        else:
            F = base_ring
            if is_FractionField(base_ring):
                if is_MPolynomialRing(base_ring.ring()) or is_PolynomialRing(base_ring.ring()):
                    f.normalize_coordinates()
                    Fn_ring = Fn.change_ring(base_ring.ring())
                    X = Fn_ring.periodic_points(n, minimal=False, formal=formal, return_scheme=True)
                    X = X.change_ring(F)
            else:
                X = f.periodic_points(n, minimal=False, formal=formal, return_scheme=True)
        print('X:', X)
        print('F:', F)
        newR = PolynomialRing(F, 'w, t', 2, order='lex')
        if not base_ring.is_field():
            ringR = PolynomialRing(base_ring, 'w, t', 2, order='lex')
        if chow:
            # create full polynomial ring
            R = PolynomialRing(F, 'v', 2*N+3, order='lex')
            var = list(R.gens())
            # create polynomial ring for result
            R2 = PolynomialRing(F, var[:N] + var[-2:])
            psi = R2.hom(N*[0]+list(newR.gens()), newR)
            # create substition to set extra variables to 0
            R_zero = {R.gen(N):1}
            for j in range(N+1, 2*N+1):
                R_zero[R.gen(j)] = 0
            t = var.pop()
            w = var.pop()
            var = var[:N]
        else:
            R = PolynomialRing(F, 'v', N+2, order='lex')
            psi = R.hom(N*[0] + list(newR.gens()), newR)
            var = list(R.gens())
            t = var.pop()
            w = var.pop()
        print('R ordering:', R.term_order())
        sigma_polynomial = 1
        # go through each affine patch to avoid repeating periodic points
        # setting the visited coordiantes to 0 as we go
        for j in range(N,-1,-1):
            Xa = X.affine_patch(j)
            print('Xa:', Xa)
            fa = Fn.dehomogenize(j)
            print('fa:', fa)
            Pa = fa.domain()
            Ra = Pa.coordinate_ring()
            # create the images for the Hom to the ring we will do the elimination over
            # with done affine patch coordinates as 0
            if chow:
                im = [R.gen(i) for i in range(j)] + (N-j)*[0] + [R.gen(i) for i in range(N, R.ngens())]
            else:
                im = list(R.gens())[:j] + (N-j)*[0] + [R.gen(i) for i in range(N, R.ngens())]
            phi = Ra.hom(R.gens()[0:len(Ra.gens())])
            # create polymomial that evaluates to the characteristic polynomial
            M = t*matrix.identity(R, N)
            g = (M-jacobian([phi(F.numerator())/phi(F.denominator()) for F in fa], var)).det()
            print('g:', g)
            # create the terms of the sigma invariants prod(w-lambda)
            g_prime = w*R(g.denominator())(im)-R(g.numerator())(im)
            print('g_prime:', g_prime)
            # move the defining polynomials to the polynomial ring
            L = [phi(h)(im) for h in Xa.defining_polynomials()]
            print('L1:', L)
            # add the appropriate final polynomial to compute the sigma invariant polynomial
            # via a Poisson product in elimination
            if chow:
                L += [g_prime + sum(R.gen(j-1)*R.gen(N+j)*(R(g.denominator())(im)) for j in range(1,N+1))]
            else:
                L += [g_prime]
            I = R.ideal(L)
            # since R is lex ordering, this is an elimination step
            print('I:', I)
            G = I.groebner_basis()
            print('G:', G)
            print('G[-1]:', G[-1])
            # the polynomial we need is the one just in w and t
            if chow:
                unflattened = G[-1]
                poly = psi(unflattened.specialization(R_zero))
                if len(list(poly)) > 0:
                    poly *= poly.coefficients()[0].inverse_of_unit()
                sigma_polynomial *= poly
            else:
                sigma_polynomial *= psi(G[-1])
        if not base_ring.is_field():
            sigma_polynomial = ringR(sigma_polynomial)
        if return_polynomial:
            return sigma_polynomial
        # if we are returing a numerical list, read off the coefficients
        # in order of degree adjusting sign appropriately
        sigmas = []
        sigma_dictionary = dict([list(reversed(i)) for i in list(sigma_polynomial)])
        degree_w, degree_t = sigma_polynomial.degrees()
        w, t = sigma_polynomial.variables()
        sigmas += [degree_w, degree_t]
        for i in range(degree_w+1):
            for j in range(degree_t+1):
                sigmas.append((-1)**(i+j)*sigma_dictionary.pop(w**(degree_w - i)*t**(degree_t - j), 0))
        return sigmas

In [46]:
R.<c> = QQ[]
P.<x,y>=ProjectiveSpace(R,1)
f=DynamicalSystem([x^2+c*y^2,y^2])
sigma(f, 2, chow=True, formal=True, return_polynomial=True)

formal: True
X: Closed subscheme of Projective Space of dimension 1 over Fraction Field of Univariate Polynomial Ring in c over Rational Field defined by:
  x^2 + x*y + (c + 1)*y^2
F: Fraction Field of Univariate Polynomial Ring in c over Rational Field
R ordering: Lexicographic term order
Xa: Closed subscheme of Affine Space of dimension 1 over Fraction Field of Univariate Polynomial Ring in c over Rational Field defined by:
  x^2 + x + c + 1
fa: Dynamical System of Affine Space of dimension 1 over Univariate Polynomial Ring in c over Rational Field
  Defn: Defined on coordinates by sending (x) to
        (x^4 + 2*c*x^2 + c^2 + c)
g: (-4)*v0^3 + (-4*c)*v0 + v4
g_prime: 4*v0^3 + 4*c*v0 + v3 - v4
L1: [v0^2 + v0 + c + 1]
I: Ideal (v0^2 + v0 + c + 1, 4*v0^3 + v0*v2 + 4*c*v0 + v3 - v4) of Multivariate Polynomial Ring in v0, v1, v2, v3, v4 over Fraction Field of Univariate Polynomial Ring in c over Rational Field
G: [v0^2 + v0 + c + 1, v0*v2 + v3 - v4 + 4*c + 4, v0*v3 - v0*v4 + (4*c + 4)*v0

w^2 + (-2)*w*t + (8*c + 8)*w + t^2 + (-8*c - 8)*t + 16*c^2 + 32*c + 16

In [44]:
f.sigma_invariants(2, formal=True)

[32, 256]

In [6]:
R.<c>=QQ[]
P.<x,y>=ProjectiveSpace(QQ,1)
f=DynamicalSystem([x^2+3*y^2,x*y])
sigma(f, 1, return_polynomial=True)

X: Closed subscheme of Projective Space of dimension 1 over Rational Field defined by:
  3*y^3
F: Rational Field
R ordering: Lexicographic term order
Xa: Closed subscheme of Affine Space of dimension 1 over Rational Field defined by:
  3
fa: Dynamical System of Affine Space of dimension 1 over Rational Field
  Defn: Defined on coordinates by sending (x) to
        ((x^2 + 3)/x)
g: (v0^2*v2 - v0^2 + 3)/v0^2
g_prime: v0^2*v1 - v0^2*v2 + v0^2 - 3
L1: [3]
I: Ideal (3, v0^2*v1 - v0^2*v2 + v0^2 - 3) of Multivariate Polynomial Ring in v0, v1, v2 over Rational Field
G: [1]
G[-1]: 1
Xa: Closed subscheme of Affine Space of dimension 1 over Rational Field defined by:
  3*y^3
fa: Dynamical System of Affine Space of dimension 1 over Rational Field
  Defn: Defined on coordinates by sending (y) to
        (y/(3*y^2 + 1))
g: (9*v0^4*v2 + 6*v0^2*v2 + 3*v0^2 + v2 - 1)/(9*v0^4 + 6*v0^2 + 1)
g_prime: v1 - v2 + 1
L1: [0]
I: Ideal (0, v1 - v2 + 1) of Multivariate Polynomial Ring in v0, v1, v2 over Rational 

w - t + 1

In [108]:
f.periodic_points(1, return_scheme=True).rational_points(F=QQbar)

[(0.500000000000000? - 1.658312395177700?*I : 1),
 (0.500000000000000? + 1.658312395177700?*I : 1),
 (1 : 0)]

In [103]:
f.multiplier_spectra(1)

[1, 1, 1]

In [3]:
R.<c>=QQ[]
P.<x,y>=ProjectiveSpace(R,1)
f=DynamicalSystem([x^2+c*y^2,y^2])
f.sigma_invariants(1, return_polynomial=True).parent()

Multivariate Polynomial Ring in w, t over Univariate Polynomial Ring in c over Rational Field

In [49]:
R.<c,d>=QQ[]
P.<x,y,z>=ProjectiveSpace(R,2)
f=DynamicalSystem([x^2+c*z^2,y^2+d*z^2,z^2])
len(dict(f.sigma_invariants(1,return_polynomial=True)))

51

In [2]:
P.<x,y,z>=ProjectiveSpace(QQ,2)
f=DynamicalSystem([x^2-z^2,y^2,z^2])
f.sigma_invariants(2,return_polynomial=True,formal = False)==f.sigma_invariants(2,return_polynomial=True,formal = True)

False

In [4]:
type(P((1,1,1)))

<class 'sage.schemes.projective.projective_point.SchemeMorphism_point_projective_field'>